# Practica 4 (Entrenamiento de redes neuronales)
### Aprendizaje automático y big data
##### _Alberto García Doménech - Pablo Daurell Marina_    
---

In [1]:
import numpy as np
from scipy.io import loadmat

In [2]:
weights = loadmat('ex4weights.mat')

theta1 = weights['Theta1'] # Shape: (25, 401)
theta2 =weights['Theta2'] # Shape: (10, 26)

In [5]:
# Cargamos los datos: 5000 ejemplos de entrenamiento (imagenes de 20x20 pixeles)
data = loadmat('ex4data1.mat')

X = data['X'] # Shape: (5000, 400)
y = data['y'].ravel() # Shape: (5000,)

num_entradas = np.shape(X)[1]
num_etiquetas = 10 # Del 0 al 9
num_ocultas = 25

# Adaptamos 'y' para usarlo en la red neuronal
y = (y - 1)
y_onehot = np.zeros((len(y), num_etiquetas)) # Shape: (5000, 10)

for i in range(len(y)):
    y_onehot[i][y[i]] = 1

In [6]:
# utilizamos de nuevo las funciones de coste,sigmoide y de gradiente
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def forward_propagation(X, theta1, theta2):
    m = X.shape[0]
    a1 = np.hstack([np.ones([m, 1]), X]) 
    z2 = np.dot(a1, theta1.T)
    a2 = np.hstack([np.ones([m, 1]), sigmoid(z2)])
    z3 = np.dot(a2, theta2.T)
    h = sigmoid(z3)
    return a1, z2, a2, z3, h

In [65]:
def backprop(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y, reg):
    '''Funcion de back-propagation para red neuronal de 3 capas'''
    
    m = X.shape[0]
    
    # Desplegamos los paramas_rn en la matrices Theta
    theta1 = np.reshape(params_rn[:num_ocultas * (num_entradas + 1)],
                       (num_ocultas, (num_entradas + 1)))
    theta2 = np.reshape(params_rn[num_ocultas * (num_entradas + 1):],
                       (num_etiquetas, (num_ocultas + 1)))
    
    # Aplicamos forward-propagation para calcular la salidas de cada capa
    a1, z2, a2, z3, h = forward_propagation(X, theta1, theta2)
    
    cost = 0
    
    for i in range(m):
        a = np.dot(-y[i,:], np.log(h[i,:]))
        b = np.dot((1-y[i,:]), np.log(1-h[i,:]))
        cost += np.sum(a - b)
        
    cost = cost/m
    cost += reg/(2*m) * (np.sum(theta1**2) + np.sum(theta2**2))
    
    return cost

In [66]:
thetaVec = np.concatenate((np.ravel(theta1), np.ravel(theta2)))
backprop(thetaVec, num_entradas, num_ocultas, num_etiquetas, X, y_onehot, 1)

0.38448779624289375